### Previous steps

In [25]:
from fastai.vision.all import *
from fastbook import *
from fastai.vision.widgets import *
import seaborn as sns


sns.set()


In [26]:
path = untar_data(URLs.MNIST_SAMPLE)


In [27]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()


In [28]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors), len(seven_tensors)


(6131, 6265)

In [29]:
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255
stacked_threes.shape


torch.Size([6131, 28, 28])

In [30]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28 * 28)

# 1 - is three and 0 - is seven
train_y = tensor([1] * len(threes) + [0] * len(sevens)).unsqueeze(1)

train_x.shape, train_y.shape


(torch.Size([12396, 784]), torch.Size([12396, 1]))

In [31]:
dset = list(zip(train_x, train_y))
x, y = dset[0]
x.shape, y


(torch.Size([784]), tensor([1]))

In [32]:
dl = DataLoader(dset, batch_size=256)
xb, yb = first(dl)
xb.shape, yb.shape


(torch.Size([256, 784]), torch.Size([256, 1]))

In [33]:
valid_3_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o))
                            for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
valid_3_tens.shape, valid_7_tens.shape


(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [34]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28 * 28)
valid_y = tensor([1] * len(valid_3_tens) + [0] *
                 len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x, valid_y))


In [35]:
valid_dl = DataLoader(valid_dset, batch_size=256)


In [36]:
lr = 1.0


In [37]:
def mnist_loss(predictions, targets):
    return torch.where(targets == 1, 1 - predictions, predictions).mean()


In [38]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()


In [46]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds > 0.5) == yb
    return correct.float().mean()


### Optimizer

nn.Linear does the same thing as our init_params and linear together. It
contains both the weights and biases in a single class

In [39]:
linear_model = nn.Linear(28*28, 1)


Every PyTorch module knows what parameters it has that can be trained; they are
available through the parameters method:

In [40]:
w, b = linear_model.parameters()
w.shape, b.shape


(torch.Size([1, 784]), torch.Size([1]))

### Create own optimizer

In [41]:
class BasicOptim:
    def __init__(self, params, lr):
        self.params = list(params)
        self.lr = lr

    def step(self, *args, **kwargs):
        for p in self.params:
            p.data -= p.grad.data * self.lr

    def zero_grad(self, *args, **kwargs):
        for p in self.params:
            p.grad = None


In [42]:
opt = BasicOptim(linear_model.parameters(), lr)


Our training loop can now be simplified:

In [43]:
def train_epoch(model):
    for xb, yb in dl:
        calc_grad(xb, yb, model)
        opt.step()
        opt.zero_grad()


In [44]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)


In [47]:
validate_epoch(linear_model)


0.483

training loop

In [49]:
def train_model(model, epochs):
    for _ in range(epochs):
        train_epoch(model)
        print(validate_epoch(model), end=' ')


In [50]:
train_model(linear_model, 20)


0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 

fastai provides the SGD class that, by default, does the same thing as our
BasicOptim

In [51]:
linear_model = nn.Linear(28*28, 1)
opt = SGD(linear_model.parameters(), lr)
train_model(linear_model, 20)


0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 

fastai also provides Learner.fit, which we can use instead of train_model.

To create a Learner, we first need to create a DataLoaders, by passing in our
training and validation DataLoaders

In [52]:
dls = DataLoaders(dl, valid_dl)


In [53]:
learn = Learner(dls,
                nn.Linear(28*28, 1),
                opt_func=SGD,
                loss_func=mnist_loss,
                metrics=batch_accuracy)


In [57]:
learn.fit(n_epoch=10, lr=lr)


epoch,train_loss,valid_loss,batch_accuracy,time
0,-9397.796875,-10425.356445,0.953386,00:00
1,-9624.174805,-10761.671875,0.953386,00:00
2,-9885.562500,-11097.989258,0.953386,00:00
3,-10169.819336,-11434.306641,0.953386,00:00
4,-10467.249023,-11770.622070,0.953386,00:00
5,-10771.580078,-12106.940430,0.953386,00:00
6,-11079.285156,-12443.255859,0.953386,00:00
7,-11388.553711,-12779.573242,0.953386,00:00
8,-11698.522461,-13115.887695,0.953386,00:00
9,-12008.795898,-13452.206055,0.953386,00:00
